- 참고문헌
  - https://github.com/ayushwattal/PDF-ChatGpt/blob/main/logic.py
  - https://www.analyticsvidhya.com/blog/2023/05/build-a-chatgpt-for-pdfs-with-langchain/
- 작동방식
  - pdf와 api키를 업로드하고 채팅을 하면 질문에 대한 pdf의 내용을 기반으로 대답을 해준다.

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !git clone https://github.com/ayushwattal/PDF-ChatGpt.git

Cloning into 'PDF-ChatGpt'...
remote: Enumerating objects: 39, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 39 (delta 15), reused 21 (delta 6), pack-reused 0
Receiving objects: 100% (39/39), 1.25 MiB | 4.46 MiB/s, done.
Resolving deltas: 100% (15/15), done.


In [9]:
# cd /content/drive/MyDrive/intern/llm_chatgpt_study/chatgpt_plus_pdf2/PDF-ChatGpt

/content/drive/MyDrive/intern/llm_chatgpt_study/chatgpt_plus_pdf2/PDF-ChatGpt


In [ ]:
cd ./PDF-ChatGpt

In [11]:
!pip install -r requirements.txt

# 웹페이지 실행

In [ ]:
!python app.py

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://4829764efc20e66890.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://4829764efc20e66890.gradio.live


In [ ]:
pwd

'/content/drive/MyDrive/intern/llm_chatgpt_study/chatgpt_plus_pdf2/PDF-ChatGpt'

# chatgpt_pdf 코드 발췌

In [12]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import PyPDFLoader
import fitz
from PIL import Image
import os
import openai
from dotenv import load_dotenv

In [31]:
pwd

'/content/drive/MyDrive/intern/llm_chatgpt_study/chatgpt_plus_pdf2/PDF-ChatGpt'

In [32]:
# Global variables
COUNT, N = 0, 0
chat_history = []
chain = ''

#api 키 설정
def set_api_key():
    """
    Sets the OpenAI API key in the environment variable.

    Returns:
        str: Message indicating the API key is set.
    """
    ##코드 수정_1114##
    load_dotenv(dotenv_path="../../../.env.local") #.env.local에 openai.api_key가 저장되어 있음. 환경변수로 사용함.
    openai.api_key=os.getenv("OPENAI_API_KEY")
    return 'OpenAI API key is set'


# 채팅 기록하기
## inputs=[chatbot, txt], outputs=[chatbot]
def add_text(history, text):
    history = history + [(text, '')]
    return history

# pdf에 기반한 chatgpt질문응답 체인 과정
## 원래는 파일을 업로드하는 것이나 colab에 실행해야 하므로 입력 파라미터를 파일이름으로 수정.
def process_file(file_name):
    # pdf파일 업로드
    loader = PyPDFLoader(file_name)
    documents = loader.load()
    # 텍스트파일 임베딩(벡터화)
    embeddings = OpenAIEmbeddings() #초기화
    pdfsearch = Chroma.from_documents(documents, embeddings) #벡터화
    # 질문응답 체인 설계
    chain = ConversationalRetrievalChain.from_llm(ChatOpenAI(temperature=0.3), #temperature이 낮을수록 더 일관된 응답 생성
                                    retriever=pdfsearch.as_retriever(search_kwargs={"k": 1}), #객체를 정보검색 모델로 변환하여 최대 1개의 검색결과 반환
                                    return_source_documents=True) #결과로는 원본문서 반환
    return chain

# 채팅 기록과 쿼리에 기반한 응답 생성
## history => inputs=[chatbot, txt, btn], outputs=[chatbot,txt]
def generate_response(history, query, btn):
    global COUNT, N, chat_history, chain

    if COUNT == 0:
        chain = process_file(btn)
        COUNT += 1

    result = chain({"question": query, 'chat_history': chat_history}, return_only_outputs=True) #질문에 대한 응답 생성
    chat_history += [(query, result["answer"])] #대화기록에 질문,응답 저장
    # N = list(result['source_documents'][0])[1][1]['page']
    print(chat_history)
    #질문에 대한 응답 합치기
    for char in result['answer']:
        history[-1][-1] += char
        yield history, ''

# Function to render a specific page of a PDF file as an image
# def render_file(file):
#     global N
#     doc = fitz.open(file.name)
#     page = doc[N]
#     # Render the page as a PNG image with a resolution of 300 DPI
#     pix = page.get_pixmap(matrix=fitz.Matrix(300/72, 300/72))
#     image = Image.frombytes('RGB', [pix.width, pix.height], pix.samples)
#     return image

In [34]:
MODEL = "gpt-3.5-turbo"
set_api_key()
file_name='../../chatgpt_plus_pdf/data/2023_GPT4All_Technical_Report.pdf'
chain=process_file(file_name)

In [ ]:
query="이 책의 저자는 누구야?"
btn=file_name
answer=generate_response(history, query, btn)
answer

<generator object generate_response at 0x7f3f794c3f40>